In [1]:
#pragma cling add_include_path("./thirdparty/libtorch/include")
#pragma cling add_include_path("./thirdparty/libtorch/include/torch/csrc/api/include")
#pragma cling add_library_path("./thirdparty/libtorch/lib")
#pragma cling load("libtorch")


#include <torch/torch.h>
#include <torch/csrc/autograd/variable.h>
#include <torch/csrc/autograd/function.h>

In [7]:
#pragma cling add_include_path("./thirdparty/nest/include")
#include <nest/nest.h>

input_line_19:1:10: fatal error: 'nest/nest.h' file not found
#include <nest/nest.h>
         ^~~~~~~~~~~~~


Interpreter Error: 

In [2]:
torch::Tensor t;
t = torch::rand({5,5}).to(torch::kFloat64)/255;

In [3]:
torch::Tensor t1 = torch::multinomial(torch::softmax(t,1),1).view({-1, 1}).neg()

In [4]:
std::cout << t1.sizes() << std::endl

[5, 1]


@0x7fb5c4d37b60

In [10]:

struct NetInput {
    torch::Tensor 
};

// Define a new Module.
struct Net : torch::nn::Module {
  Net(int64_t num_actions, observation_shape, bool_t use_lstm) { // TODO
    // Construct and register two Linear submodules.
    fc1 = register_module("fc1", torch::nn::Linear(784, 64));
    fc2 = register_module("fc2", torch::nn::Linear(64, 32));
    fc3 = register_module("fc3", torch::nn::Linear(32, 10));
  }

  // Implement the Net's algorithm.
  torch::Tensor forward(NetInput inputs, torch::Tensor& core_state) {
      
        torch::Tensor x;
        x = inputs.frame; // TODO check
        T = x.sizes()[0];
        B = x.sizes()[1];
        x = x.flatten(0,1);
        x = x.to(torch::kFloat64);

        x = torch::relu(fc1); // TODO

        x = x.view({T*B,-1});
        x = torch::relu(fc(x)); // TODO

        torch::Tensor one_hot_last_action = torch::one_hot(
            inputs.last_action.view({T*B}),
            num_actions
        );
      
        torch::Tensor clipped_reward = inputs.reward
                .clamp(-1,1)
                .view({T*B,1});
      
        torch::Tensor core_input = torch::cat({
            x,
            clipped_reward,
            one_hot_last_action
        },-10);
      
        torch::Tensor core_output;
        if (use_lstm){
            core_output = core_input.view({T,B,-10});
            core_output_list = ();
            torch::Tensor not_done = inputs.
                                     done.
                                     neg().
                                     to(torch::kFloat64);
            
            
            for () {
                // Reset core state to zero whenever an episode ended.
                // Make `done` broadcastable with (num_layers, B, hidden_size)
                // states:
                core_state = 
            };            
            
            core_output = torch::cat(core_output).view({0,1}); //TODO
        } else {
            core_output = core_input;
        };
      
        torch::Tensor policy_logits = (); // TODO
        torch::Tensor baseline = (); // TODO
      
        torch::Tensor action;
        if (self.is_training()) {
            action = torch::multinomial(
                torch::softmax(policy_logits, 1),
                1);
        } else {
            action = torch.argmax(policy_logits,1);
        };
   
        policy_logits = policy_logits.view({T,B,num_actions});
        baseline = baseline.view({T,B});
        action = action.view({T,B});

        return x; // TODO return tuple
  }

  // Use one of many "standard library" modules.
  torch::nn::Linear fc1{nullptr}, fc2{nullptr}, fc3{nullptr};
};



In [11]:
int main() {
  // Create a new Net.
  auto net = std::make_shared<Net>();

  // Create a multi-threaded data loader for the MNIST dataset.
  auto data_loader = torch::data::make_data_loader(
      torch::data::datasets::MNIST("./data").map(
          torch::data::transforms::Stack<>()),
      /*batch_size=*/64);

  // Instantiate an SGD optimization algorithm to update our Net's parameters.
  torch::optim::SGD optimizer(net->parameters(), /*lr=*/0.01);

  for (size_t epoch = 1; epoch <= 10; ++epoch) {
    size_t batch_index = 0;
    // Iterate the data loader to yield batches from the dataset.
    for (auto& batch : *data_loader) {
      // Reset gradients.
      optimizer.zero_grad();
      // Execute the model on the input data.
      torch::Tensor prediction = net->forward(batch.data);
      // Compute a loss value to judge the prediction of our model.
      torch::Tensor loss = torch::nll_loss(prediction, batch.target);
      // Compute gradients of the loss w.r.t. the parameters of our model.
      loss.backward();
      // Update the parameters based on the calculated gradients.
      optimizer.step();
      // Output the loss and checkpoint every 100 batches.
      if (++batch_index % 100 == 0) {
        std::cout << "Epoch: " << epoch << " | Batch: " << batch_index
                  << " | Loss: " << loss.item<float>() << std::endl;
        // Serialize your model periodically as a checkpoint.
        torch::save(net, "net.pt");
      }
    }
  }
}

In [12]:
main()

Standard Exception: Error opening images file at ./data/train-images-idx3-ubyte
Exception raised from read_images at ../torch/csrc/api/src/data/datasets/mnist.cpp:66 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >) + 0x69 (0x7fbac1801569 in /home/thorad/Core/Projects/Qenv/notebooks/thirdparty/libtorch/lib/libc10.so)
frame #1: <unknown function> + 0x33615cc (0x7fbaa668c5cc in /home/thorad/Core/Projects/Qenv/notebooks/thirdparty/libtorch/lib/libtorch_cpu.so)
frame #2: torch::data::datasets::MNIST::MNIST(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, torch::data::datasets::MNIST::Mode) + 0x46 (0x7fbaa668e136 in /home/thorad/Core/Projects/Qenv/notebooks/thirdparty/libtorch/lib/libtorch_cpu.so)
frame #3: [0x7fbac9585075]
frame #4: [0x7fbac95bc031]
frame #5: cling::IncrementalExecutor::executeWrapper(llvm::StringRef, cling::Value*) const + 0x38a 